In [1]:
#librerias necesarias
import os
import pandas as pd
import numpy as np
import faiss
from faiss import write_index
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
from config import CFG

c:\Users\pablo\anaconda3\envs\APLN\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_processed_data(with_na=False, n_samples=None):
    # List the files in the processed_data directory
    files = os.listdir('dataset/processed_data_2')

    # Read the files into a dataframe
    for idx, file in enumerate(files):
        if idx == 0:
            df = pd.read_csv('dataset/processed_data_2/' + file, na_values=['', ' ', 'No information found.'])
        else:
            df = pd.concat([df, pd.read_csv('dataset/processed_data_2/' + file, na_values=['', ' ', 'No information found.'])], ignore_index=True)
    
    if not with_na:
        df = df.dropna()

    if n_samples is not None:
        df = df.sample(n_samples)

    return df

df = read_processed_data(with_na = CFG.with_na, n_samples=CFG.n_samples)

In [3]:
df

,question,question_id,question_type,answer,focus,id,source,url,cui,semanticType,semanticGroup
0,What is (are) A guide to clinical trials for c...,0000001-1,information,"If you have cancer, a clinical trial may be an...",A guide to clinical trials for cancer,1,ADAM,https://www.nlm.nih.gov/medlineplus/ency/patie...,C0006826,T191,Disorders
6,What is (are) A1C test ?,0000003-1,information,A1C is a lab test that shows the average level...,A1C test,3,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0456984,T033,Disorders
8,What is (are) Aarskog syndrome ?,0000004-1,information,Aarskog syndrome is a very rare disease that a...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
9,What causes Aarskog syndrome ?,0000004-2,causes,Aarskog syndrome is a genetic disorder that is...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
10,What are the symptoms of Aarskog syndrome ?,0000004-3,symptoms,Symptoms of this condition include: Belly butt...,Aarskog syndrome,4,ADAM,https://www.nlm.nih.gov/medlineplus/ency/artic...,C0175701,T019,Disorders
...,...,...,...,...,...,...,...,...,...,...,...
46972,What is (are) Von Willebrand Disease ?,0000139-1,information,Von Willebrand disease (VWD) is a bleeding dis...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46973,What causes Von Willebrand Disease ?,0000139-2,causes,Von Willebrand disease (VWD) is almost always ...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46974,What are the symptoms of Von Willebrand Disease ?,0000139-3,symptoms,The signs and symptoms of von Willebrand disea...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders
46975,How to diagnose Von Willebrand Disease ?,0000139-4,exams and tests,Early diagnosis of von Willebrand disease (VWD...,Von Willebrand Disease,139,NHLBI,http://www.nhlbi.nih.gov/health/health-topics/...,C0042974,T047,Disorders


In [4]:
df.keys()

Index(['question', 'question_id', 'question_type', 'answer', 'focus', 'id',
       'source', 'url', 'cui', 'semanticType', 'semanticGroup'],
      dtype='object')

In [5]:
class TextDataset(Dataset):
    def __init__(self, df):# Input is a pandas dataframe
        self.questions = df.question.tolist()
        self.question_ids = df.question_id.tolist()
        self.question_types = df.question_type.tolist()
        self.answers = df.answer.tolist()
        self.focus = df.focus.tolist()
        self.doc_id = df.id.tolist()
        self.source = df.source.tolist()
        self.url = df.url.tolist()
        self.cui = df.cui.tolist()
        self.semantic_type = df.semanticType.tolist()
        self.semantic_group = df.semanticGroup.tolist()
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return {'Q': self.questions[idx], 
                'Q_id': self.question_ids[idx], 
                'Q_T': self.question_types[idx], 
                'A': self.answers[idx],
                'F': self.focus[idx],
                'D_id': self.doc_id[idx],
                'S': self.source[idx],
                'U': self.url[idx],
                'C': self.cui[idx],
                'S_T': self.semantic_type[idx],
                'S_G': self.semantic_group[idx]}
    

def collate_fn(batch, tokenizer=AutoTokenizer.from_pretrained(CFG.embedding_model)):
    # Extrae las preguntas de los elementos del batch
    questions = [item['Q'] for item in batch]
    
    # Tokeniza las preguntas en un lote
    tokenized_questions = tokenizer(
        questions,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=512
    )
    
    # No hay necesidad de usar pad_sequence aquí, ya que tokenizer maneja el padding
    return {
        "input_ids": tokenized_questions['input_ids'],
        "attention_mask": tokenized_questions['attention_mask']
    }



def get_bert_embeddings(ds, batch_size=CFG.batch_size):
    dataloader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    model = AutoModel.from_pretrained(CFG.embedding_model)
    model = model.to(CFG.device)
    model.eval()
    embeddings = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(CFG.device)
            attention_mask = batch['attention_mask'].to(CFG.device)
            outputs = model(input_ids, attention_mask)
            last_hidden_state = outputs.last_hidden_state
            cls_embedding = last_hidden_state[:, 0, :]
            embeddings.append(cls_embedding.cpu().numpy())
    return np.concatenate(embeddings)


documents = TextDataset(df)

In [6]:
# Función para crear el índice FAISS
def create_faiss_index(embeddings):
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  return index

embeddings = get_bert_embeddings(documents, CFG.batch_size)
index = create_faiss_index(embeddings)# Crea el índice FAISS con los embeddings

100%|██████████| 238/238 [00:17<00:00, 13.25it/s]


In [7]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [8]:
# Función para obtener los embeddings de una consulta de texto
def get_query_embedding(query_text, device = CFG.device):
    tokenizer = AutoTokenizer.from_pretrained(CFG.embedding_model)
    model = AutoModel.from_pretrained(CFG.embedding_model).to(device)
    inputs = tokenizer(query_text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    query_embedding = outputs.last_hidden_state.mean(1).squeeze().cpu().numpy()
    return query_embedding

# Ejemplo de consulta
query_text = "abdominal mass ?"
query_embedding = get_query_embedding(query_text)
query_vector = np.expand_dims(query_embedding, axis=0)
# print(f'Query vector: {query_vector}')
# Realiza la búsqueda en el índice FAISS
D, I = index.search(query_vector, k=10)  # Busca los 5 documentos más similares

print("Consulta realizada con el texto:", query_text)
print("Documentos más similares:")
for i, idx in enumerate(I[0], start=1):
    print(f"{i}: Documento {idx} con una distancia de {D[0][i-1]}")
    print("Contenido del documento:", documents[idx]['Q'])
    print("Respuesta obtenida:", documents[idx]['A'].replace('\n', ' ') )
    # Devolver el answer de cada pregunta

Consulta realizada con el texto: abdominal mass ?
Documentos más similares:
1: Documento 33 con una distancia de 9.776694297790527
Contenido del documento: What causes Abdominal mass ?
Respuesta obtenida: Several conditions can cause an abdominal mass: Abdominal aortic aneurysm  can cause a pulsating mass around the navel.,  Bladder distention  (urinary bladder over-filled with fluid) can cause a firm mass in the center of the lower abdomen above the pelvic bones. In extreme cases, it can reach as far up as the navel.,  Cholecystitis  can cause a very tender mass that is felt below the liver in the right-upper quadrant (occasionally).,  Colon cancer  can cause a mass almost anywhere in the abdomen.,  Crohn disease  or bowel obstruction can cause many tender, sausage-shaped masses anywhere in the abdomen.,  Diverticulitis  can cause a mass that is usually located in the left-lower quadrant., Gallbladder tumor can cause a tender, irregularly shaped mass in the right-upper quadrant.,  Hyd